### リスト
P: 製品のリスト
M: 原料のリスト

### 定数
stock_${m}$(m∈M): 原料mの在庫量

require_${p,m}$(p∈P, m∈M): 製品pを生産するのに必要な原料mの量

gain_${p}$(p∈P): 製品pを生産した際の利得

### 変数
x_${p}$(p∈P): 製品pの生産量

### 制約式
x_${p}$>=0(p∈P): 製品pの生産量は0以上

生産は在庫の範囲で行う

### 目的関数
gain_${p}$を最大化する

### データのインポート

In [2]:
import pandas as pd
import pulp

In [3]:
stock_df = pd.read_csv('stocks.csv')
stock_df

,m,stock
0,m1,35
1,m2,22
2,m3,27


In [4]:
require_df = pd.read_csv('requires.csv')
require_df

,p,m,require
0,p1,m1,2
1,p1,m2,0
2,p1,m3,1
3,p2,m1,3
4,p2,m2,2
5,p2,m3,0
6,p3,m1,0
7,p3,m2,2
8,p3,m3,2
9,p4,m1,2


In [5]:
gain_df = pd.read_csv('gains.csv')
gain_df

,p,gain
0,p1,3
1,p2,4
2,p3,4
3,p4,5


### リストの定義

In [7]:
P = gain_df['p'].tolist()
print(P)

['p1', 'p2', 'p3', 'p4']


In [9]:
M = stock_df['m'].tolist()
print(M)

['m1', 'm2', 'm3']


### 定数の定義

In [11]:
stock = {row.m: row.stock for row in stock_df.itertuples()}
print(stock)

{'m1': 35, 'm2': 22, 'm3': 27}


In [12]:
stock = dict(zip(stock_df['m'], stock_df['stock']))
print(stock)

{'m1': 35, 'm2': 22, 'm3': 27}


In [14]:
stock = dict((row.m, row.stock) for row in stock_df.itertuples())
print(stock)

{'m1': 35, 'm2': 22, 'm3': 27}


In [15]:
stock = stock_df.set_index('m').to_dict()['stock']
print(stock)

{'m1': 35, 'm2': 22, 'm3': 27}


In [16]:
require = {(row.p, row.m): row.require for row in require_df.itertuples()}
print(require)

{('p1', 'm1'): 2, ('p1', 'm2'): 0, ('p1', 'm3'): 1, ('p2', 'm1'): 3, ('p2', 'm2'): 2, ('p2', 'm3'): 0, ('p3', 'm1'): 0, ('p3', 'm2'): 2, ('p3', 'm3'): 2, ('p4', 'm1'): 2, ('p4', 'm2'): 2, ('p4', 'm3'): 2}


In [17]:
gain = {row.p: row.gain for row in gain_df.itertuples()}
print(gain)

{'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}


### 線形計画問題の定義

In [18]:
problem = pulp.LpProblem('LP2', pulp.LpMaximize)

### 変数の定義

In [19]:
x = pulp.LpVariable.dicts('x', P, cat='Continuous')

### 制約式の定義

In [20]:
for p in P:
    problem += x[p] >= 0

In [21]:
for m in M:
    problem += pulp.lpSum([require[p,m] * x[p] for p in P]) <= stock[m]

### 目的関数の定義

In [22]:
problem += pulp.lpSum([gain[p] * x[p] for p in P])

In [23]:
status = problem.solve()
print('Status: ', pulp.LpStatus[status])

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Sep 26 2023 

command line - /usr/local/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/arm64/cbc /tmp/89e2527f6ac64d53a8f39ac3a28d3075-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/89e2527f6ac64d53a8f39ac3a28d3075-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 30 RHS
At line 38 BOUNDS
At line 43 ENDATA
Problem MODEL has 7 rows, 4 columns and 13 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-4) rows, 4 (0) columns and 9 (-4) elements
0  Obj -0 Dual inf 17.499996 (4)
4  Obj 80.428571
Optimal - objective value 80.428571
After Postsolve, objective 80.428571, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 80.42857143 - 4 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):

In [24]:
for p in P:
    print(p, x[p].value())

print('obj=', problem.objective.value())

p1 12.142857
p2 3.5714286
p3 7.4285714
p4 0.0
obj= 80.42857099999999


### 生産量が整数値で得られるように数理最適化モデルを書き換える

In [25]:
# 変数の定義
x = pulp.LpVariable.dict('x', P, cat='Integer')

In [26]:
problem = pulp.LpProblem('IP', pulp.LpMaximize)

In [2]:
import pandas as pd
import pulp

# データの取得
require_df = pd.read_csv('requires.csv')
stock_df = pd.read_csv('stocks.csv')
gain_df = pd.read_csv('gains.csv')

# リストの定義
P = gain_df['p'].tolist()
M = stock_df['m'].tolist()

# 定数の定義
stock = {row.m:row.stock for row in stock_df.itertuples()}
gain = {row.p:row.gain for row in gain_df.itertuples()}
require = {(row.p,row.m):row.require for row in require_df.itertuples()}

# 数理最適化
problem = pulp.LpProblem('IP', pulp.LpMaximize)

# 変数の定義
x = pulp.LpVariable.dicts('x', P, cat='Integer')

# 制約式の定義
for p in P:
    problem += x[p] >= 0
for m in M:
    problem += pulp.lpSum([require[p,m] * x[p] for p in P]) <= stock[m]

# 目的関数の定義
problem += pulp.lpSum([gain[p] * x[p] for p in P])

# 求解
status = problem.solve()
print('Status:', pulp.LpStatus[status])

# 計算結果の表示
for p in P:
    print(p, x[p].value())

print('obj=', problem.objective.value())

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Sep 26 2023 

command line - /usr/local/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/arm64/cbc /tmp/6bfa6bf61e884735a04f84eec2a8fbca-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/6bfa6bf61e884735a04f84eec2a8fbca-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 38 RHS
At line 46 BOUNDS
At line 51 ENDATA
Problem MODEL has 7 rows, 4 columns and 13 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 80.4286 - 0.00 seconds
Cgl0004I processed model has 3 rows, 4 columns (4 integer (0 of which binary)) and 9 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 76 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0038I Full problem 3 rows 4 columns, reduced to 3 rows 3 columns
Cbc0012I Integer solution of 79 found by 